# Imports


TODO:
1) check value/vat/no vat
2) check headlines, fix it


In [ ]:
# !export GPN_DB_HOST=192.168.10.36
LIMIT = 5000

USE_CONTROL_SET = True

In [ ]:
try:
    LIMIT = int(LIMIT)
except:
    LIMIT = 5000
        
 
print(f'{LIMIT=}')



COLLECTION_NAME = 'documents'
if USE_CONTROL_SET:
    COLLECTION_NAME = 'documents_temp'
    
print(f'{COLLECTION_NAME=}')

In [ ]:
IMPORT_FRESH_ONLY = True # re-import all if False

In [ ]:
import logging
import platform
import sys
import os

import tensorflow as tf

logger = logging.getLogger('export_ts')
logger.setLevel(logging.DEBUG)


ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(levelname)s - %(asctime)s - %(name)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.debug('--=logging started=--')

print("tf",tf.__version__)
CPU = platform.processor()
print (f'Running on CPU:{CPU}')



In [ ]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
import analyser.hyperparams 


### Imports...

In [ ]:
%matplotlib inline

#3rd party--------
import json
import re
import pandas as pd
import numpy as np

from pandas import DataFrame
from pathlib import Path

from datetime import datetime
from datetime import date

from math import log1p
from pandas import DataFrame


from pathlib import Path

from bson import ObjectId
from bson import json_util


from dateutil.relativedelta import relativedelta

from pymongo import ASCENDING
 

import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display, Markdown

from collections import Counter



#ours---------

 
from analyser.persistence import DbJsonDoc
from analyser.structures import DocumentState
from analyser.legal_docs import LegalDocument, PARAGRAPH_DELIMITER, GenericDocument, Paragraph, embedd_tokens
from analyser.ml_tools import SemanticTag
from analyser.headers_detector import get_tokens_features

from integration.db import get_mongodb_connection
from integration.word_document_parser import join_paragraphs

from colab_support.renderer import plot_embedding

from tf_support.embedder_elmo import ElmoEmbedder
from tf_support.super_contract_model import get_semantic_map_new, t_semantic_map_keys_price
from tf_support.super_contract_model import semantic_map_keys_contract, t_semantic_map_keys_common, t_semantic_map_keys_org

from trainsets.retrain_contract_uber_model import save_contract_data_arrays



In [ ]:
def get_doc_from_db_by_id (objid):
    db = get_mongodb_connection()
    documents_collection = db[COLLECTION_NAME]
    _id = ObjectId(str(objid))
    jdata = documents_collection.find_one({'_id': _id})
    return jdata
    # return DbJsonDoc(jdata)





### Init ml flow

In [ ]:
import mlflow
 
ml_flow_url = os.environ.get('MLFLOW_URL', "http://192.168.10.38:5000")
mlflow.set_tracking_uri(ml_flow_url)
print(f'{ml_flow_url=}', 'set MLFLOW_URL env var to re-define')
mlflow.set_experiment("Анализатор: экспорт данных из mongo")
             

### Prepare workdir

In [ ]:
_work_dir_default = Path(analyser.hyperparams.__file__).parent.parent.parent / 'work'
work_dir = os.environ.get('GPN_WORK_DIR', _work_dir_default)

if not os.path.isdir(work_dir):
    os.mkdir(work_dir)

analyser.hyperparams.work_dir = work_dir
reports_path = Path(analyser.hyperparams.__file__).parent.parent / 'training_reports/'


assert os.path.isdir(analyser.hyperparams.work_dir)

In [ ]:


NOTEBOOKS_DIR = str(Path(analyser.__file__).parent.parent/'trainsets')
print(f'{LIMIT=}')
print(f'{NOTEBOOKS_DIR=}')
print(f'{analyser.hyperparams.work_dir=}', )
print(f'{reports_path=}', )

### Query DB for contact IDs

#### Load meta data CSV

In [ ]:
contract_trainset_meta_fn = 'contract_trainset_meta.temp.csv'
if USE_CONTROL_SET:
    contract_trainset_meta_fn = 'contract_trainset_meta.control.temp.csv'
    
export_fn = str(Path(work_dir) / contract_trainset_meta_fn)

In [ ]:
!touch {export_fn}
# !tail {export_fn}

In [ ]:


# stats = pd.read_csv(export_fn, index_col=0)
mlflow.log_param('metafile',  export_fn)

try:
    stats = pd.read_csv(export_fn, index_col=0)
    if not 'analyze_date' in stats:
        stats['analyze_date'] = None
        
    if not 'documentType' in stats:
        stats['documentType'] = 'Unknown'
    
    stats['user_correction_date'] = pd.to_datetime(stats['user_correction_date'], utc=True)
    stats['analyze_date'] = pd.to_datetime(stats['analyze_date'], utc=True)

    lastdate = stats[["user_correction_date", 'analyze_date']].max().max()
    
    mlflow.log_param('last_doc_export_date',  lastdate)
    mlflow.log_metric('docs_in_metafile_start',  len(stats))
    
except Exception as ex:
    print(f'ERROR: cannot load {export_fn}')
    logger.exception(ex)
    logger.error(f'cannot load {export_fn}')

    lastdate = datetime(1900, 1, 1)
    stats = DataFrame()
    # stats = stats.astype({'currency':str})
    stats['currency']=''
    
if not IMPORT_FRESH_ONLY:
    lastdate = datetime(1900, 1, 1)
    
print(f'lastdate={lastdate} ; export_fn={export_fn}')
stats['source'] = 'db'


In [ ]:
 
display(Markdown(f"#### {lastdate:%d.%m.%Y} -- дата последнего экспортированного документа"))


# functions

In [ ]:

def get_amount(attr_tree):
  _value_tag = attr_tree.get('price')
  amount = None
  if _value_tag is not None:
    amount = _value_tag.get('amount_netto')
    if amount is None:
      amount = _value_tag.get('amount_brutto')
    if amount is None:
      amount = _value_tag.get('amount')
  return amount

        
    
def add_stats_record(d: DbJsonDoc, stats: DataFrame, source='db'):
  _id = str(d.get_id())
    
  attr_tree = d.get_attributes_tree()
 

  stats.at[_id, 'checksum'] = d.get_tokens_for_embedding().get_checksum()
  stats.at[_id, 'version'] = d.get_version_string()
  stats.at[_id, 'documentType'] = d.documentType
    
 
  stats.at[_id, 'source'] = source
  stats.at[_id, 'export_date'] = datetime.now()
  stats.at[_id, 'len'] = len(d)
  stats.at[_id, 'analyze_date'] = pd.to_datetime( d.analysis['analyze_timestamp'] , utc=True)


  n_headers = len(d.analysis.get('headers', []))
  stats.at[_id, 'headers'] = n_headers
    
  _value_tag = attr_tree.get('price')
  
  stats.at[_id, 'value'] = None
  stats.at[_id, 'value_log1p'] = None
  stats.at[_id, 'value_span'] = None
  stats.at[_id, 'currency'] = None
    
  if _value_tag is not None:
    amount = get_amount(attr_tree) 
    if amount:
        
        stats.at[_id, 'value'] = amount.get('value') 
        stats.at[_id, 'value_log1p'] = log1p(amount.get('value') )
        stats.at[_id, 'value_span'] = amount.get('span', [0,0]) [0]
    
#         print( stats.at[_id, 'value'])
    
    stats.at[_id, 'currency'] = _value_tag.get('currency', {}).get('value')
  
  _orgs = attr_tree.get('orgs', [{},{}]) 
  if len(_orgs)>0:
      if _orgs[0]:
          stats.at[_id, 'org-1-name'] = _orgs[0].get('name', {}).get('value')
          stats.at[_id, 'org-1-alias'] = _orgs[0].get('alias', {}).get('value')
          stats.at[_id, 'org-1-type'] = _orgs[0].get('type', {}).get('value')

      if len(_orgs)>1:
          if _orgs[1]:
            stats.at[_id, 'org-2-name'] = _orgs[1].get('name', {}).get('value')
            stats.at[_id, 'org-2-alias'] = _orgs[1].get('alias', {}).get('value')
            stats.at[_id, 'org-2-type'] = _orgs[1].get('type', {}).get('value')

  stats.at[_id, 'subject'] = attr_tree.get('subject', {}).get('value')
  stats.at[_id, 'subject confidence'] = attr_tree.get('subject', {}).get('confidence')
    

  span = attr_tree.get('subject', {}).get('span',[0,0])
  subject_len = span[1] - span[0]
  stats.at[_id, 'subj_len'] = subject_len  

  stats.at[_id, 'user_correction_date'] = None
  if d.user is not None:
    # if 'attributes_tree' in d.user and 'creation_date' in d.user['attributes_tree']:
    # stats.at[_id, 'user_correction_date'] = d.user['attributes_tree']['creation_date']
    stats.at[_id, 'user_correction_date'] = pd.to_datetime(d.user[ 'updateDate'], utc=True)

    

    # find_in_dict('attributes_tree.creation_date', d.user)

  valid_state = (DocumentState.Excluded.value==d.state or DocumentState.Done.value==d.state)
  if not valid_state:
    print(_id, 'invalid state: ', d.state)
  valid_struct = ('contract' in d.analysis['attributes_tree'])
  if not valid_struct:
    print(_id, 'invalid structure: ', d.state)
  stats.at[_id, 'valid'] = valid_state and valid_struct and subject_len>0 and subject_len<=300 and n_headers>0


  


# d = get_doc_by_id(ObjectId('5fe34f62b770574a005553be'))
# jd = DbJsonDoc(d)
# add_stats_record(jd, stats)
# stats.loc['5fe34f62b770574a005553be']

# load legacy json

In [ ]:
_DEBUG = False

if not USE_CONTROL_SET:

    file_data=None

    try:
        fn = work_dir / 'documents.json'
        with open(fn) as file:
            file_data = json.load(file, object_hook=json_util.object_hook)

            print(f'total docs in {fn} is {len(file_data)}')    
  
    except Exception as ex:
        logger.exception(ex)    

    if _DEBUG:
        a_doc_from_json=DbJsonDoc(file_data[3])

        print(a_doc_from_json.get_attributes_tree())
        print(a_doc_from_json.get_version_string())

In [ ]:

# TODO: import!!!



print("semantic_map_keys_contract", semantic_map_keys_contract)

 

if _DEBUG:
    sm_test = get_semantic_map_new(a_doc_from_json)
    plot_embedding(sm_test[0:150], f'get_semantic_map_new: semantic map')
    print(sm_test['date-begin'].max())
    print(sm_test['date-begin'])

In [ ]:
_DEBUG

In [ ]:
#test one
from analyser.headers_detector import get_tokens_features

if _DEBUG:
    d = get_doc_from_db_by_id(ObjectId('636dfd1473925c8dae26f910'))
    if d is not None:
        jd = DbJsonDoc(d)
        semantic_map: DataFrame = get_semantic_map_new(jd)

        tokens_features = get_tokens_features(jd.get_tokens_map_unchaged().tokens)

        plot_embedding(semantic_map, f'get_semantic_map_new: semantic map')
        plot_embedding(tokens_features, f'tokens')

In [ ]:
# tokens_features

### Adding data from old json file to the stats table

In [ ]:
# fff
files_dict = {}
files_cnt = 0

if not USE_CONTROL_SET:
    
    if file_data:
        k=0
        for d in file_data:
            k+=1 
            jd = DbJsonDoc(d)
        #     print( jd.analysis['analyze_timestamp'])
            files_dict[jd.get_id()] = jd
            try:            
                add_stats_record(jd, stats, source = 'file')
                files_cnt+=1
            except Exception as e:
                print(e)
                print(jd.get_id())
    #             raise (e)

            if k % 100 == 0:

                print(f'{k} of {len(file_data)}')
                stats.to_csv(export_fn, index=True)
                print(f'stats saved to {export_fn}')


    mlflow.log_param('files_in_json',  files_cnt)            
    stats.to_csv(export_fn, index=True)
 


print(f'stats saved to {export_fn}')

# del file_data
stats




In [ ]:
# stats = stats.drop('63c506afe2456d59975e0fcd')
stats[stats.index.duplicated()]



In [ ]:
if not USE_CONTROL_SET:
    invalid = stats[stats['valid']==False]
    print(f'{len(invalid)}: number of invalid records')
    mlflow.log_metric('invalid records',  len(invalid))     
    invalid

# Fetch fresh docs from Mongo DB

In [ ]:

# from datetime import timedelta
# if True:

lastdate  += relativedelta(days=-15)  
# lastdate = datetime.combine(lastdate, datetime.min.time())
# lastdate = pd.to_datetime( lastdate , utc=True)
lastdate

In [ ]:
report_date = datetime.now() + relativedelta(days=-2)  
report_date

In [ ]:
yesterday = datetime.now() + relativedelta(days=-1)  
# {'analysis.analyze_timestamp': {'$gt': yesterday}},




 
query = {'user.updateDate': {'$gt': report_date}}

db = get_mongodb_connection()
documents_collection = db[COLLECTION_NAME]

sorting = [('analysis.analyze_timestamp', ASCENDING), ('user.updateDate', ASCENDING)]
res = documents_collection.find(filter=query, 
                                sort=sorting,
                                projection={'_id': True, 'user.updateDate':True, 'state':True, 'parse.documentType':True}
                               ).limit(LIMIT)

_r = list([i for i in res])
# print(res[0])
_s = f"#### {len(_r)} -- всего размечено документов после {report_date:%d.%m.%Y}"
mlflow.log_metric('fresh_user_docs_in_db',  len(_r))     
mlflow.log_param('db_docs_collection',  COLLECTION_NAME)     
display(Markdown(_s))
yseterday_ids=[i["_id"] for i in _r]


In [ ]:


if USE_CONTROL_SET:
    query = {
      '$and': [
        {"parse.documentType":{ '$in': ["AGREEMENT", "CONTRACT", "SUPPLEMENTARY_AGREEMENT"] }  },
        { 'subset': 'CONTROL_TRAIN'},
 
        {'$or': [
              {"analysis.attributes_tree": {"$ne": None}},
              {"user.attributes_tree": {"$ne": None}}
            ]},
 
        {'$or': [
          {'analysis.analyze_timestamp': {'$gt': lastdate}}, 
          {'user.updateDate': {'$gt': lastdate}}
        ]}
      ]
    }
    
 
    
else:
    query = {
      '$and': [
        {"parse.documentType":{ '$in': ["AGREEMENT", "CONTRACT", "SUPPLEMENTARY_AGREEMENT"] }  },

 
        {'$or': [
              {"analysis.attributes_tree": {"$ne": None}},
              {"user.attributes_tree": {"$ne": None}}
            ]},
 
        {'$or': [
          {'analysis.analyze_timestamp': {'$gt': lastdate}}, 
          {'user.updateDate': {'$gt': lastdate}}
        ]}
      ]
    }

db = get_mongodb_connection()
documents_collection = db[COLLECTION_NAME]
sorting = [('analysis.analyze_timestamp', ASCENDING), ('user.updateDate', ASCENDING)]
res = documents_collection.find(filter=query, 
                                sort=sorting,
                                projection={'_id': True, 'user.updateDate':True, 'state':True, 'parse.documentType':True}
#                                             'analysis.attributes_tree.version': True,
#                                             'analysis.attributes_tree.contract.subject': True}
                               ).limit(LIMIT)

res = list([i for i in res])
mlflow.log_metric('fresh_docs_in_db',  len(res))     
# print(res[0])
_s = f"#### {len(res)} -- всего проанализировано новых документов после {lastdate:%d.%m.%Y}"
display(Markdown(_s))

res[:4]

# Prepare training set

In [ ]:
new_docs_ids = docs_ids = [i["_id"] for i in res]
print(f"{len(new_docs_ids)=}")


In [ ]:
stats.columns

In [ ]:
def add_to_stats_list(docs_ids, stats):
    for k, oid in enumerate(docs_ids):
        d = get_doc_from_db_by_id( str(oid))
        jd = None
        try:
            jd = DbJsonDoc(d)            
        except Exception as e:
            print(f'missing {oid=}')
            stats = stats.drop(index=oid)
            logger.exception (e)
        
        if jd:
            add_stats_record(jd, stats)
            
        if k % 100 == 0:
            print(f'{k} of {len(docs_ids)}')
            stats.to_csv(export_fn, index=True)
            print(f'stats saved to {export_fn}')
        
   


add_to_stats_list(new_docs_ids, stats)     

stats.to_csv(export_fn, index=True)
print(f'stats saved to {export_fn}')
mlflow.log_metric('docs_in_metafile_result',  len(stats))


In [ ]:
stats

## Fixing missing Headers [legacy]

In [ ]:
s = stats[ stats['source']=='db'] 
s =s[s['headers'].isna()]
 
_ids = list(s.index)
print(len(_ids))
 
del _ids
s

In [ ]:
stats = stats[stats.valid!=False]
len(stats)

### Put all to lower

In [ ]:
stats2 = stats.copy()

if False:
    for i in (1,2):
        for key in ['alias', 'name', 'type']:
            k=f'org-{i}-{key}'
            stats2[k] = stats2[k].str.lower()
        
        
# stats2['org-1-alias'] = stats2['org-1-alias'].str.lower()
# stats2['org-1-name'] = stats2['org-1-name'].str.lower()
# stats2['org-2-name'] = stats2['org-2-name'].str.lower()

# stats2['org-1-type'] = stats2['org-1-type'].str.lower()
# stats2['org-2-type'] = stats2['org-2-type'].str.lower()
stats2

# Statistics
### Collecting org types

In [ ]:
stats2['subject'].value_counts()

In [ ]:
stats2['org-1-alias'].value_counts()

In [ ]:
stats2['org-2-alias'].value_counts()

In [ ]:
score_column_name = "score"

In [ ]:


def calc_no_value_importance(stats2, column_name, score_k=2.):
        
    for i, row in stats2.iterrows():
        if pd.isna(row[column_name]):
            stats2.at[i, score_column_name] *= score_k
                
def calc_user_importance(stats2):
        
    for i, row in stats2.iterrows():
        if not pd.isna(row['unseen']):

            if row['unseen'] == True:
                stats2.at[i, score_column_name] *= 2
                
def calc_val_importance(stats2):
 
    for i, row in stats2.iterrows():
        if not pd.isna(row['value']):
            val = row['value']
            if val >= 2000:
                stats2.at[i, score_column_name] *= 1./log1p(val)
            if val < 2000:
                stats2.at[i, score_column_name] *= 4.
        else:
            stats2.at[i, score_column_name] *= 10.0 #error
            
def calc_column_val_importance(stats2, column_name, k=1.):
    counts = stats2[column_name].value_counts()
 
        
    for i, row in stats2.iterrows():
        val = row[column_name]
        if val in counts:
            count = counts[val]
            stats2.at[i, score_column_name] *= k * log1p(count)
#             print(count, log1p(count),  1./log1p(count))
        else:
            stats2.at[i, score_column_name] *= 10.0 #error

            
for i, row in stats2.iterrows():        
  stats2.at[i, 'url'] = f'http://gpn-audit.nemosoft.ru/#/audit/edit/{i}'

    
stats2 ['unseen'] = pd.isna( stats2['user_correction_date'])

stats2[score_column_name] = 1.
 

calc_no_value_importance(stats2, 'org-1-alias', 2.)
calc_no_value_importance(stats2, 'org-2-alias', 2.)
calc_no_value_importance(stats2, 'org-1-name', 3.)
calc_no_value_importance(stats2, 'org-2-name', 3.)
calc_no_value_importance(stats2, 'user_correction_date', 10.)
calc_no_value_importance(stats2, 'subject', 4.)
calc_no_value_importance(stats2, 'value', 5.)

## calc_column_val_importance(stats2, 'org-1-name')
## calc_column_val_importance(stats2, 'org-2-name')

calc_column_val_importance(stats2, 'org-1-alias')
calc_column_val_importance(stats2, 'org-2-alias')
calc_column_val_importance(stats2, 'subject', k = 3.)

calc_user_importance(stats2)
calc_val_importance(stats2)

# drop_duplicates(subset=['checksum']).
stats2 = stats2.sort_values([score_column_name], ascending=True)
print(len(stats2))
stats2.head()

In [ ]:
# sub = stats2.sort_values(['score'], ascending=True).copy()
# sub

## Find and remove duplicates

In [ ]:
stats2['len'] //= 10 #similar lens
stats2['len'] *= 10 #similar lens
stats2.sort_values(['len', 'org-1-name', 'org-2-name', 'org-1-alias','org-2-alias', 'value', 'subject'], ascending=False)

In [ ]:
for i, row in stats2.iterrows():
  _str = ':'.join(sorted([str(x) for x in
                          [row['len'], row['org-1-alias'], row['org-2-alias'], row['org-1-name'], row['org-2-name'],
                           row['value'], row['subject'], row['headers']  ]]))
  stats2.at[i, 'hash'] = hash(_str)

stats2 = stats2.sort_values(['hash'])
print(len(stats2))
stats2

In [ ]:
unks = stats2['hash'].value_counts()

for i, row in stats2.iterrows():   
    stats2.at[i, 'dups'] = unks[row['hash']]
    
dups = stats2.sort_values(['dups', 'hash'], ascending=False)
# dups.to_csv(Path(work_dir) / 'contract_trainset_meta.duplicates.csv', index=True)

# dups[40:].head(40)
len(dups)

In [ ]:
# dups.duplicated(subset=['hash'])

### Drop duplicates

In [ ]:
dups_s = dups.sort_values([score_column_name], ascending=True).copy()

dups_s = dups_s.drop_duplicates(subset=['hash'])
# dups_s = dups_s.sort_values([score_column_name], ascending=True)
# dups_s = dups_s.sort_values([score_column_name], ascending=True)
# dups_s = dups_s[dups_s[score_column_name] < 5000]
print(len(dups_s))

# dups_s [dups_s['len'] > 30000].head(40)

mlflow.log_metric('duplicates',  len(stats) - len(dups_s))
mlflow.log_metric('train_set',  len(dups_s))

## Remove rare subjects and very long docs

In [ ]:
plt.figure(figsize=(12, 4))
ax = dups_s[dups_s['len'] < 30000] ['len'].plot.hist(bins=20, alpha=0.8)
ax.set_title('Text Length')
plt.show()

In [ ]:
print(dups_s[score_column_name].min())
plt.figure(figsize=(12, 4))
ax = dups_s[score_column_name].plot.hist(bins=28, alpha=0.8)
ax.set_title('Distibution by score')
plt.show()

In [ ]:

dups_s['valid'] = True

print("excluding too long docs")
for i, row in dups_s.iterrows():   
    if row['len'] > 30000:
        print ('excluding', i, row[0], row['subject'], row['len'])
        dups_s.at[i, 'valid'] = False  
        
dups_s = dups_s[dups_s['valid'] == True]

print("excluding docs with rare subj")
cnts = dups_s ['subject'].value_counts()
for i, row in dups_s.iterrows():   
    if not pd.isna(row['subject']):
        if cnts[row['subject']] < 3:
            print ('excluding', i, row[0], row['subject'])
            dups_s.at[i, 'valid'] = False

dups_s = dups_s[dups_s['valid'] == True]  

In [ ]:
%matplotlib inline
try:
    cnt = dups_s['subject'].value_counts()

    plt.figure(figsize=(12, 6 ))
    sns.barplot(x=cnt.values, y=cnt.index)

    print(  cnt )

    plt.title('Frequency Distribution of subjects')
    plt.xlabel('Number of Occurrences')
    plt.show()
except:
    pass


In [ ]:
dups_s.index.name = '_id'
_fn = Path(work_dir) / 'contract_trainset_meta.csv'
dups_s.to_csv(_fn, index=True)
mlflow.log_artifact(_fn)


_s = f"### {len(dups_s)} - Всего новых документов в обучающем наборе на {lastdate:%d.%m.%Y}"
display(Markdown(_s))


dups_s

In [ ]:
dups_s[dups_s['unseen']].to_csv(Path(work_dir) / 'contract_trainset_meta.to_markup.csv', index=True)

In [ ]:
dups_s[dups_s['unseen']]

In [ ]:
%%time


query = {
  '$and': [
    {"parse.documentType":{ '$in': ["AGREEMENT", "CONTRACT", "SUPPLEMENTARY_AGREEMENT"] }  },      
    {"user.attributes_tree": {"$ne": None}},
#       {'user.updateDate': {'$gt': lastdate}}    
  ]
}

db = get_mongodb_connection()
documents_collection = db[COLLECTION_NAME]
sorting = [('analysis.analyze_timestamp', ASCENDING), ('user.updateDate', ASCENDING)]
res = documents_collection.find(filter=query, 
                                sort=sorting,
                                projection={'_id': True, 'user.updateDate':True, 'state':True, 'parse.documentType':True}
#                                             'analysis.attributes_tree.version': True,
#                                             'analysis.attributes_tree.contract.subject': True}
                               ).limit(5000)

res = list([i for i in res])

_s = f"#### {len(res)} -- Всего документов, которые размечены биочеловеком 🤦"
display(Markdown(_s))


# 🤦 Find markup errors

In [ ]:
user_docs_ids  = [i["_id"] for i in res]
print('user_docs_ids', len(user_docs_ids))


In [ ]:
errors_report_file_prefix = "user_markup_errors"
errors_report_metric_prefix = "user"

user_docs_ids = user_docs_ids
%run -i -t {NOTEBOOKS_DIR}/validate_markup.ipynb
del user_docs_ids

In [ ]:




try:
    ax = userdocs.groupby(
        [userdocs["дата редактирования"].dt.year, userdocs["дата редактирования"].dt.month ])[
            ['errors count']].count().plot(kind="bar", figsize=(16,3))

    plt.show()
except Exception as e:
    logger.exception(e)
    pass

In [ ]:

# auto_docs = validate_markup(auto_docs_ids)


# Statistics
### Collecting org types

In [ ]:

statistics = {}


l = list(userdocs['Форма собственности 1'].values) + list( userdocs['Форма собственности 2'].values)
l= [str(k) for k in l]
c = Counter(l)


statistics['org_types_cased'] = pd.DataFrame(c.items(),  columns = ['type', 'count']).sort_values(['count', 'type'], ascending=False)
statistics['org_types_cased'].to_csv("statistics.org_types_cased.csv")
statistics['org_types_cased']

In [ ]:
l = list(userdocs['Форма собственности 1'].values) + list( userdocs['Форма собственности 2'].values)
l= [str(k).lower() for k in l]
c = Counter(l)


statistics['org_types'] = pd.DataFrame(c.items(),  columns = ['type', 'count']).sort_values(['count', 'type'], ascending=False)
statistics['org_types'].to_csv("statistics.org_types.csv")
statistics['org_types']

### Aliases

In [ ]:
userdocs['subject kind']

In [ ]:
def count_alias_pairs(cased:bool):
    _df = pd.DataFrame()
    for i, v in userdocs.iterrows():
        if cased:
            key = ' & '.join(   np.sort( list([ str(v['Псевдоним 1']), str(v['Псевдоним 2'])])) ) 
        else:
            key = ' & '.join(   np.sort( list([ str(v['Псевдоним 1']).lower(), str(v['Псевдоним 2']).lower()])) ) 
        key2 = ' & '.join( [str(v['subject kind']), key ] )
        _df.at[i, 'count'] = key2



    _t = pd.DataFrame(_df['count'].value_counts())
    for i, v in _t.iterrows():
        s, a1, a2 = str(i).split('&')
    #     print(a2)
        _t.at[i, 'alias 1'] = a1
        _t.at[i, 'alias 2'] = a2
        _t.at[i, 'subject'] = s
#     statistics['alias_pairs']=_t
#     statistics['alias_pairs'].to_csv("statistics.alias_pair.csv")
#     _t
    return _t


statistics['alias_pairs']=count_alias_pairs(cased=False)
statistics['alias_pairs'].to_csv("statistics.alias_pair.csv")

statistics['alias_pairs_cased']=count_alias_pairs(cased=True)
statistics['alias_pairs_cased'].to_csv("statistics.alias_pair_cased.csv")

In [ ]:
statistics['alias_pairs_cased']

In [ ]:
statistics['alias_pairs']

## Validate auto markup

In [ ]:
del userdocs
auto_docs_ids = stats[ (stats.user_correction_date.isnull()) & (stats.source=='db') & (stats['documentType']=='CONTRACT') ].index
errors_report_file_prefix = "ai_markup_errors"
errors_report_metric_prefix = "ai"
user_docs_ids = auto_docs_ids
%run -i -t {NOTEBOOKS_DIR}/validate_markup.ipynb
del user_docs_ids

auto_docs = userdocs

### Invalidate no-date docs

In [ ]:
bad_docs = auto_docs[ (auto_docs['Дата'].isnull()) | (auto_docs['errors severity']>=10)]
bad_docs.index

for i, row in stats.iterrows():
    if i in bad_docs.index:
        stats.at[i, 'valid'] = False
# stats.loc[bad_docs.index].valid=False
stats.loc[bad_docs.index].valid
# for i, row in stats.iterrows():
#     if i

In [ ]:
good_docs = auto_docs[ (~auto_docs['Дата'].isnull()) & (auto_docs['errors severity']<10)]
good_docs
# auto_docs[auto_docs['errors count']<3]

# Make datapoints

In [ ]:
work_dir

In [ ]:



embedder = ElmoEmbedder.get_instance('elmo')  # lazy init




if False: #TODO clean it up, this code moved to .py
    def _dp_fn(doc_id, suffix):
        return os.path.join(work_dir, 'datasets', f'{doc_id}-datapoint-{suffix}.npy')


    def save_contract_data_arrays(db_json_doc: DbJsonDoc):
        # TODO: trim long documens according to contract parser

        id_ = db_json_doc.get_id()

        tokens_map: TextMap = db_json_doc.get_tokens_for_embedding()

        # 1) EMBEDDINGS
        print(len(tokens_map))
        embeddings = embedd_tokens(tokens_map,
                                   embedder,
                                   log_key=f'id={id_} chs={tokens_map.get_checksum()}')

        # 2) TOKEN FEATURES
        token_features: DataFrame = get_tokens_features(db_json_doc.get_tokens_map_unchaged().tokens)

        # 3) SEMANTIC MAP
        semantic_map: DataFrame = get_semantic_map_new(db_json_doc)
        #####

        np.save(_dp_fn(id_, 'token_features'), token_features)
        np.save(_dp_fn(id_, 'semantic_map'), semantic_map)
        _embeddings_file = _dp_fn(id_, 'embeddings')
        np.save(_embeddings_file, embeddings)
        print(f'embeddings saved to {_embeddings_file} {embeddings.shape}')

In [ ]:
_docs_ids = [str(i) for i in new_docs_ids]
intersection_set = set.intersection(set(list(dups_s.index)), set(_docs_ids)) 
len(intersection_set)

In [ ]:
# good_docs.index

# intersection_set = set.intersection(set(list(dups_s.index)), set(_docs_ids)) 

### Trainset fixtures
#### Invalidate long subjects

In [ ]:
%%time

ids = list(dups_s.index)


if not 'subject_pos' in stats:
    stats['subject_pos'] = None
        
ids_of_long_subject_docs=[]

for k, oid in enumerate(ids):

    _id = str(oid)
    iiid= ObjectId(_id)
    
    src = dups_s.loc[_id]['source']
    jd = None
    if src == 'db':
#       if False:
        d = get_doc_from_db_by_id(iiid)
        if d is not None:
            jd = DbJsonDoc(d)
            
            if jd.state not in [15,12]:
                print(jd.state, iiid)
            
            stats.at[_id, 'documentType'] = jd.documentType
            dups_s.at[_id, 'documentType'] = jd.documentType
            
            attr_tree = jd.get_attributes_tree()  
            span = attr_tree.get('subject', {}).get('span',[0,0])
            subject_len = span[1]-span[0]
            stats.at[_id, 'subj_len'] = subject_len
            dups_s.at[_id, 'subj_len'] = subject_len  
            
            stats.at[_id, 'subject_pos'] = span[0]
            dups_s.at[_id, 'subject_pos'] = span[0]  
                
            if span[0]==0:
                stats.at[_id, 'valid'] = False
                dups_s.at[_id, 'valid'] = False
                
            if subject_len >= 200:
                
                stats.at[_id, 'valid'] = False
                dups_s.at[_id, 'valid'] = False
                
                
                
                ids_of_long_subject_docs.append(oid)
                
                tm = jd.get_tokens_map_unchaged()
                quote = tm.text_range(span)
                print(k, iiid, span, span[1]-span[0])
#                 print('-'*100)
                sentence_span = tm.sentence_at_index(span[0])
                quote2 = tm.text_range(sentence_span)
                print(quote2[:50])
                print('='*100)
# stats[stats['documentType']=='CONTRACT']

dups_s.to_csv(Path(work_dir) / 'contract_trainset_meta.csv', index=True)

#### Fix missing headers

In [ ]:
%%time




def save_analysis(db_document: DbJsonDoc, doc_dummie: LegalDocument) -> DbJsonDoc:
  documents_collection = get_mongodb_connection()[COLLECTION_NAME]
  
  _analyse_json_obj: dict = doc_dummie.to_json_obj()
  print("--analyse_json_obj['headers'] len=", len(_analyse_json_obj['headers']))

#   to_save = db_document.as_dict()
#   to_save['headers'] = _analyse_json_obj['headers']

  documents_collection.update_one({'_id': doc_dummie.get_id()}, {'$set': {'analysis.headers': _analyse_json_obj['headers']}})
 
    
  print("updated", doc_dummie.get_id())


def restore_headers (response, oid) :
  doc = join_paragraphs(response, oid)
  return doc
   


def restore_headlines(k, oid):    

    iiid= ObjectId(oid)
    src = dups_s.loc[oid]['source']
    jd = None
    if src == 'db':
#       if False:
        d = get_doc_from_db_by_id(iiid)
        if d is not None:
            jd = DbJsonDoc(d)
            
            if jd.state in [15,12]:
                if len(jd.analysis.get('headers', [])) > 0:
                    #  all is ok
#                     print(oid)
                    pass
                else:
                     doc_with_paragraphs = restore_headers(jd.parse, iiid)
                     save_analysis(jd,  doc_with_paragraphs)
                     print ('--', k, oid, len(doc_with_paragraphs.paragraphs))

                    
for k, oid in enumerate(ids):
    restore_headlines(k,oid)


In [ ]:
# restore_headlines(1,'62d149474ae369b4976e31fc'): 

In [ ]:
len(ids_of_long_subject_docs)

In [ ]:
dups_s

In [ ]:
stats.to_csv(export_fn, index=True)
mlflow.log_artifact(export_fn)

# Main sequence

In [ ]:
%%time

from IPython.display import clear_output

# raise('test mode!')

# enumerate(list(intersection_set))
for k, oid in enumerate(ids): 
# for k, oid in enumerate(list(ids)):
    if k % 20 == 0:
        clear_output(wait=True)
        
# 
    iiid= ObjectId(oid)
    src = dups_s.loc[oid]['source']
    jd = None
    if src == 'db':
#       if False:
        d = get_doc_from_db_by_id(iiid)
        if d is not None:
          jd = DbJsonDoc(d)
    else:
#         if False:
        print(oid, k, 'of', len(ids), src)
        jd = files_dict[iiid]
        
    print(oid, k, 'of', len(ids), src)

    if jd is not None:
        if jd.state in [15,12]:                
            save_contract_data_arrays(jd)


In [ ]:
print('see results at')
print(f'{mlflow.get_registry_uri()}/#/experiments/{mlflow.last_active_run().info.experiment_id}/runs/{mlflow.last_active_run().info.run_id}')